# Sample End-to-End Machine Learning Project

In [1]:
# Objective : To predict the median housing price in any distict
# Notes :: 
# 1) I only include essential codes that are necessary for the execution of the ML algorithmn
# 2) Codes that are data explortary in nature are excluded and examined in another file 

In [2]:
import pandas as pd 
import numpy as np

housing = pd.read_csv(r"C:\Users\tanzh\Documents\AI & Machine Learning\Hands-On Machine Learning Codes\Datasets\housing.csv")

In [3]:
housing.head(10)

,longitude,latitude,housing_median_age,total_rooms,total_bedrooms,population,households,median_income,median_house_value,ocean_proximity
0,-122.23,37.88,41.0,880.0,129.0,322.0,126.0,8.3252,452600.0,NEAR BAY
1,-122.22,37.86,21.0,7099.0,1106.0,2401.0,1138.0,8.3014,358500.0,NEAR BAY
2,-122.24,37.85,52.0,1467.0,190.0,496.0,177.0,7.2574,352100.0,NEAR BAY
3,-122.25,37.85,52.0,1274.0,235.0,558.0,219.0,5.6431,341300.0,NEAR BAY
4,-122.25,37.85,52.0,1627.0,280.0,565.0,259.0,3.8462,342200.0,NEAR BAY
5,-122.25,37.85,52.0,919.0,213.0,413.0,193.0,4.0368,269700.0,NEAR BAY
6,-122.25,37.84,52.0,2535.0,489.0,1094.0,514.0,3.6591,299200.0,NEAR BAY
7,-122.25,37.84,52.0,3104.0,687.0,1157.0,647.0,3.1200,241400.0,NEAR BAY
8,-122.26,37.84,42.0,2555.0,665.0,1206.0,595.0,2.0804,226700.0,NEAR BAY
9,-122.25,37.84,52.0,3549.0,707.0,1551.0,714.0,3.6912,261100.0,NEAR BAY


# Creating Training and Test datasets

In [4]:
# we will be using a technique called stratified sampling 
# to avoid sampling bias, the training and test data sets must be representative of the overall population
# suppose MEDIAN INCOME is an important attribute that explains MEDIAN PRICES of houses
# we then want to ensure that data composition of the training and test datasets, using MEDIAN INCOME as the criteria, is representative of the overall population 

In [5]:
# we first create a calculated attribute called income cat

housing["income_cat"] = pd.cut(housing["median_income"], bins=[0.,1.5,3.0,4.5,6.,np.inf], labels= [1,2,3,4,5])

In [6]:
housing.loc[0:20,["income_cat", "median_income"]]

,income_cat,median_income
0,5,8.3252
1,5,8.3014
2,5,7.2574
3,4,5.6431
4,3,3.8462
5,3,4.0368
6,3,3.6591
7,3,3.1200
8,2,2.0804
9,3,3.6912


In [7]:
from sklearn.model_selection import StratifiedShuffleSplit

split = StratifiedShuffleSplit(n_splits=1, test_size=0.2, random_state=42)
# n_splits refer to the Number of re-shuffling & splitting iterations.

for train_index, test_index in split.split(housing, housing["income_cat"]) : 
    strat_train_set = housing.loc[train_index]
    strat_test_set = housing.loc[test_index]

# both the training and test datasets has income category proportion almost identical to the full dataset

In [8]:
# We then remove the income_cat column from the training and test datasets

for set_ in (strat_train_set, strat_test_set) : 
    set_.drop("income_cat", axis = 1, inplace = True)

# axis = 1 refer to the column items, if equal to 0, it will refer to the row items
# inplace = True means that the removal take place at the variable level


# Preparing a clean dataset

In [9]:
# Now we want to seperate the dependent variable and the independent variables

housing = strat_train_set.drop("median_house_value", axis = 1) # the drop method creates a copy of the data and does not impact strat_train_set
housing_labels = strat_train_set["median_house_value"].copy() # this only contain the dependent variable [median_house_value]

# NOTE THAT HOUSING VARIABLE NOW ONLY CONTAIN THE TRAINING SET

# 1) Data Cleaning - Numercial Attributes 

In [10]:
# most ML cannot work well with missing attributes. we have the following options 
#1 get rid of the feature/column with missing entries 
#2 set the missing entries values to some values (0, median, mean etc)

In [11]:
# we first create a copy of the data without categorical/text attributes as the technique that we will be using can only be applied on numerical attributes 
housing_num = housing.drop("ocean_proximity", axis=1) # this creates a copy of the data without the categorical columns

# we will be using a technique called SimpleImputer which allow us to fill in the missing entries with some strategy (mean/median/etc)
from sklearn.impute import SimpleImputer
imputer = SimpleImputer(strategy= "median") # define the strategy to a variable 

In [12]:
imputer.fit(housing_num) # this will compute the median of each column attributes and store the results in its STATISTICS_instance variable

SimpleImputer(add_indicator=False, copy=True, fill_value=None,
              missing_values=nan, strategy='median', verbose=0)

In [13]:
imputer.statistics_

array([-118.51  ,   34.26  ,   29.    , 2119.5   ,  433.    , 1164.    ,
        408.    ,    3.5409])

In [14]:
# Before we transform, glance at the number of missing entries in total_bedroom
# there are 16,354 entries in the total_bedrooms column while there are 16,512 entires for the rest of the columns
housing_num.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 16512 entries, 17606 to 15775
Data columns (total 8 columns):
longitude             16512 non-null float64
latitude              16512 non-null float64
housing_median_age    16512 non-null float64
total_rooms           16512 non-null float64
total_bedrooms        16354 non-null float64
population            16512 non-null float64
households            16512 non-null float64
median_income         16512 non-null float64
dtypes: float64(8)
memory usage: 1.1 MB


In [15]:
# we now want to use the trained imputer to transform the training set by replacing the missing vaues with the learned medians
X = imputer.transform(housing_num) # this will return plain NumPy array

In [16]:
# to put it back into a pandas dataframe
housing_tr = pd.DataFrame(X, columns=housing_num.columns, index=housing_num.index)
housing_tr.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 16512 entries, 17606 to 15775
Data columns (total 8 columns):
longitude             16512 non-null float64
latitude              16512 non-null float64
housing_median_age    16512 non-null float64
total_rooms           16512 non-null float64
total_bedrooms        16512 non-null float64
population            16512 non-null float64
households            16512 non-null float64
median_income         16512 non-null float64
dtypes: float64(8)
memory usage: 1.1 MB


# 2) Data Cleaning - Text / Categorical Attrbutes

In [17]:
# we first create a column that contains only the attributes with categorical/text attributes
housing_cat = housing[["ocean_proximity"]] 

In [18]:
# we will be using a technique called one-hot encoding which create one binary attribute per category
from sklearn.preprocessing import OneHotEncoder
cat_encoder = OneHotEncoder()

housing_cat_1hot = cat_encoder.fit_transform(housing_cat) # note that we are fitting the training data here

In [19]:
housing_cat_1hot # the output is a SciPy sparse matrix where only the 1s are stored

<16512x5 sparse matrix of type '<class 'numpy.float64'>'
	with 16512 stored elements in Compressed Sparse Row format>

# Feature Scaling

In [20]:
# with few exceptions, most ML algorithmns do not perform well when the input attributes have very different numercial scales
# there are two common methods to scaling

# 1) Min-Max Scaling 

In [21]:
# also know as normalisation
# values are shifted such that it will range-bound between 0 and 1 
# this technique will be imapcted by extreme values on both ends
# formula : [x(i) - min(x)] / [max(x) - min(x)]
# the transformer in SL is known as MinMaxScaler

# 2) Standardisation 

In [22]:
# this technique does not range-bound the values which may pose a problem for some ML algorithmns
# this technique is also less imapcted by extreme values on both ends
# formula : [x(i) - mean(x)] / std(x)
# the transformer in SL is known as StandardScaler

# Transformation Pipeline for Numerical Attributes

In [23]:
# For all transformation, only fit the scaler to the training data only

from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler

num_pipeline = Pipeline([
                        ("imputer", SimpleImputer(strategy="median")),
                        ("std_scaler", StandardScaler()),
                        ])

# Transformation Pipeline for Categorical Attributes & Full Pipeline

In [24]:
from sklearn.compose import ColumnTransformer

num_attribute = list(housing_num) # housing_num is the variable with the missing entries under the column < total_bedroooms > 
cat_attribute = ["ocean_proximity"]

full_pipeline = ColumnTransformer([
                                  ("num", num_pipeline, num_attribute), # tjis will return a dense matrix
                                  ("cat", OneHotEncoder(), cat_attribute), # this will return a sparse matric
                                  ])

In [25]:
housing_prepared = full_pipeline.fit_transform(housing) #housing is the amended dataset that contain only the input features

# Training and Evaluating on Training Dataset (Linear Regression)

In [26]:
# the first model we are testing on is the Linear Regression Model

from sklearn.linear_model import LinearRegression

lin_reg = LinearRegression() # we are assigning the linear regression function to a variable 

lin_reg.fit(housing_prepared, housing_labels) # the parameter is the [Dependent Variable, Independent Variable] # this is for training the model
# this fit the training data and the labels using linear regression
# after this step, we will have a working linear regression model

LinearRegression(copy_X=True, fit_intercept=True, n_jobs=None, normalize=False)

# Testing on samples from the training set

In [27]:
some_data = housing.iloc[:5] # to assign some data points from the training set
some_labels = housing_labels[:5] # to assign some data points from the solutions, i.e. the dependent variable
some_data_prepared = full_pipeline.transform(some_data)
some_data_prediction = lin_reg.predict(some_data_prepared)

print(some_data_prediction)

[211574.39523833 321345.10513719 210947.519838    61921.01197837
 192362.32961119]


In [28]:
print(list(some_labels))

[286600.0, 340600.0, 196900.0, 46300.0, 254500.0]


# RMSE on the Linear Regression model

In [29]:
from sklearn.metrics import mean_squared_error
housing_prediction = lin_reg.predict(housing_prepared) # making prediction, the parameter is the training dataset
lin_mse = mean_squared_error(housing_labels,housing_prediction) # the parameters are solutions and predictions
lin_rmse = np.sqrt(lin_mse) # rmse is derived by taking the square root of mean square error (MSE)
print(lin_rmse)

69050.98178244587


In [30]:
# the prediction error is $69,050 
# this is an example of a model underfitting the model

# Training and Evaluating on Training Dataset (Decision Tree Regressor)

In [31]:
from sklearn.tree import DecisionTreeRegressor

tree_reg = DecisionTreeRegressor()
tree_reg.fit(housing_prepared, housing_labels) # this line is for training the model

DecisionTreeRegressor(criterion='mse', max_depth=None, max_features=None,
                      max_leaf_nodes=None, min_impurity_decrease=0.0,
                      min_impurity_split=None, min_samples_leaf=1,
                      min_samples_split=2, min_weight_fraction_leaf=0.0,
                      presort=False, random_state=None, splitter='best')

In [32]:
housing_prediction_tree = tree_reg.predict(housing_prepared) # making prediction
tree_mse = mean_squared_error(housing_labels, housing_prediction_tree) # generating the mse using the dependent variable and the predictions
tree_rmse = np.sqrt(tree_mse) 
print(tree_rmse)

0.0


In [33]:
# we have overfitted the model very badly

# Cross-Validation on Decision Tree Regressors

In [34]:
# one way to evaluate the Decision Tree Model will be to use the train_test_split() function to split the training dataset into two seperate datasets
# the first set is a smaller dataset from the inital training dataset, and the second set is the validation set
# we then train the model using the full training data set

In [35]:
# an alternative method is technique called the k-fold cross-validation 
#1 the code split the training dataset into x distinct subsets (folds)
#2 it train and evaluate the Decision Tree model x times
#3 in each of the x iteration, it pick a different fold for evaluation while training on other 9 folds
#4 the result is an array containing the 10 evaluation scores

In [36]:
# we will be using the k-fold cross validation technique below 

from sklearn.model_selection import cross_val_score
scores = cross_val_score(tree_reg, housing_prepared, housing_labels,scoring = "neg_mean_squared_error", cv = 10) # this generates 10 MSE scores
# cv refer to the number of folds

tree_rmse_scores = np.sqrt(-scores) # this generates 10 RMSE scores

In [37]:
def display_scores_stats(lst) : 
    print("min is " + str(min(lst)))
    print("max is " + str(max(lst)))

    print("mean is " + str(np.mean(lst)))
    print("std is " +  str(np.std(lst)))

In [38]:
print(tree_rmse_scores)

[66960.64859997 66083.01931244 72665.52826216 69775.96933289
 68721.7404781  76383.98569033 67568.71729512 68764.37811774
 71566.8303965  69198.41115606]


In [39]:
# it seems that the Decision Tree Model perform worse than the Linear Regression Model

In [40]:
display_scores_stats(tree_rmse_scores)

min is 66083.01931244174
max is 76383.98569032672
mean is 69768.92286413105
std is 2900.453046702138


# Cross-Validation on Linear Regression

In [41]:
scores_lin = cross_val_score(lin_reg, housing_prepared, housing_labels, scoring = "neg_mean_squared_error", cv = 10) 
scores_lin_rmse = np.sqrt(-scores_lin)

display_scores_stats(scores_lin_rmse)

min is 65361.14176205049
max is 74639.88837894418
mean is 69223.18594556303
std is 2657.268311277693


# Cross-Validation on Random Forest Model

In [42]:
from sklearn.ensemble import RandomForestRegressor
forest_reg = RandomForestRegressor()
forest_reg.fit(housing_prepared, housing_labels) # this is to train the model

RandomForestRegressor(bootstrap=True, criterion='mse', max_depth=None,
                      max_features='auto', max_leaf_nodes=None,
                      min_impurity_decrease=0.0, min_impurity_split=None,
                      min_samples_leaf=1, min_samples_split=2,
                      min_weight_fraction_leaf=0.0, n_estimators=10,
                      n_jobs=None, oob_score=False, random_state=None,
                      verbose=0, warm_start=False)

In [43]:
forest_scores = cross_val_score(forest_reg, housing_prepared, housing_labels, scoring= "neg_mean_squared_error", cv = 10)
forest_scores_rmse = np.sqrt(-forest_scores)

display_scores_stats(forest_scores_rmse)

min is 48923.992301484315
max is 55515.12827198866
mean is 52252.68040376816
std is 1801.5139882157175


In [44]:
# Random Forest work by training many Decision Tree models on random subsets of features and averaging out their predictions

In [45]:
forest_scores_rmse

array([50841.80882396, 48923.99230148, 50874.63485223, 52531.57090896,
       51960.62775258, 55515.12827199, 52723.49175231, 54075.72096227,
       53758.11090236, 51321.71750952])

# Fine-Tuning The Models - GridSearch

In [46]:
# you define which parameters you want to experiment and the values to try out
# it will then use cross-validation to evaluate all the possible combinations of hyperparameter values

In [47]:
from sklearn.model_selection import GridSearchCV

param_grid = [ {'n_estimators' : [3,10,30], 'max_features' : [2,4,6,8]}, 
               {'bootstrap' : [False], 'n_estimators' : [3,10], 'max_features': [2,3,4]},
             ]

# in the first dict, there are 12 combinations --> first group of specification
# in the second dict, there are 6 combinations --> second group of specification

# n_estimators and max_features are hyperparameters specified in the first dict
# param_grid evaluate all 3 * 4 = 12 combinations of n_estimators and max_features hyperparameter values specified in the first dict
# then, param_grid evaluate all 2 * 3 = 6 combinations of n_estimators and max_features hyperparameter values in the second dict, but with the bootstrap parameter set to False instead of True (which is the default value for this parameter)

# In two iteration, there will be a total of 18 combination of hyperparameters

In [48]:
forest_reg = RandomForestRegressor()

grid_search = GridSearchCV(forest_reg, param_grid, cv = 5, scoring = "neg_mean_squared_error",return_train_score = True)
# the grid search will explore the 18 combinations of RandomForestRegressor hyperparameter values and train each model 5 times 
# in total, there will be 90 rounds of trainings

grid_search.fit(housing_prepared, housing_labels)

GridSearchCV(cv=5, error_score='raise-deprecating',
             estimator=RandomForestRegressor(bootstrap=True, criterion='mse',
                                             max_depth=None,
                                             max_features='auto',
                                             max_leaf_nodes=None,
                                             min_impurity_decrease=0.0,
                                             min_impurity_split=None,
                                             min_samples_leaf=1,
                                             min_samples_split=2,
                                             min_weight_fraction_leaf=0.0,
                                             n_estimators='warn', n_jobs=None,
                                             oob_score=False, random_state=None,
                                             verbose=0, warm_start=False),
             iid='warn', n_jobs=None,
             param_grid=[{'max_features': [2, 4, 6, 8],


In [49]:
grid_search.best_params_ # this return the best combination of parameters

{'max_features': 8, 'n_estimators': 30}

In [50]:
grid_search.best_estimator_ # this return the best estimator

RandomForestRegressor(bootstrap=True, criterion='mse', max_depth=None,
                      max_features=8, max_leaf_nodes=None,
                      min_impurity_decrease=0.0, min_impurity_split=None,
                      min_samples_leaf=1, min_samples_split=2,
                      min_weight_fraction_leaf=0.0, n_estimators=30,
                      n_jobs=None, oob_score=False, random_state=None,
                      verbose=0, warm_start=False)

In [51]:
cvres = grid_search.cv_results_

for mean_score, params in zip(cvres["mean_test_score"], cvres["params"]) : 
    print(np.sqrt(-mean_score), params)

63835.43014109018 {'max_features': 2, 'n_estimators': 3}
55653.13133902398 {'max_features': 2, 'n_estimators': 10}
52786.928567892784 {'max_features': 2, 'n_estimators': 30}
60513.446655517495 {'max_features': 4, 'n_estimators': 3}
53178.19451752226 {'max_features': 4, 'n_estimators': 10}
50596.91200747191 {'max_features': 4, 'n_estimators': 30}
59201.38025643767 {'max_features': 6, 'n_estimators': 3}
52657.67378613478 {'max_features': 6, 'n_estimators': 10}
50529.40189773309 {'max_features': 6, 'n_estimators': 30}
58596.514006475496 {'max_features': 8, 'n_estimators': 3}
52498.35753988611 {'max_features': 8, 'n_estimators': 10}
50455.36662471176 {'max_features': 8, 'n_estimators': 30}
62922.027106291716 {'bootstrap': False, 'max_features': 2, 'n_estimators': 3}
54040.77958889584 {'bootstrap': False, 'max_features': 2, 'n_estimators': 10}
60074.322098235956 {'bootstrap': False, 'max_features': 3, 'n_estimators': 3}
52464.39397477669 {'bootstrap': False, 'max_features': 3, 'n_estimators

# Randomized Search

In [52]:
# the grid search approach is fine when you are exploring relatively few combinations of hyperparameters
# when the hyperparameters search space is large, it is better to use RandomizedSearchCV instead

# Instead of trying out all the possible combination, RandomizedSearchCV evaluate a given number of random combination by selecting a random value for each hyperparameter at each iteration 
# if you let the RandomizedSearchCV run for 1000 iterations, this approach will explore 1000 different values for each hyperparameters instead of just a few values for each hyperparameters with the grid search method

# Analyzing the best models and their errors

In [53]:
# the RandomForestRegressor can indicate the relative importance of each attribute for making accurate predictions

feature_importances = grid_search.best_estimator_.feature_importances_
feature_importances

array([1.14964944e-01, 1.02753179e-01, 5.10712766e-02, 2.88390604e-02,
       2.73757360e-02, 3.59971155e-02, 2.43326732e-02, 4.45753633e-01,
       1.10645301e-02, 1.48474815e-01, 5.51048534e-05, 1.98750350e-03,
       7.33042970e-03])

In [54]:
cat_encoder = full_pipeline.named_transformers_["cat"]
cat_one_hot_attribs = list(cat_encoder.categories_[0])
attributes = num_attribute + cat_one_hot_attribs

sorted(zip(feature_importances, attributes), reverse = True)

[(0.44575363301049215, 'median_income'),
 (0.14847481473535173, 'INLAND'),
 (0.11496494350292082, 'longitude'),
 (0.10275317883521921, 'latitude'),
 (0.0510712766306358, 'housing_median_age'),
 (0.03599711550984855, 'population'),
 (0.028839060438268605, 'total_rooms'),
 (0.02737573595727476, 'total_bedrooms'),
 (0.024332673198226448, 'households'),
 (0.011064530130259124, '<1H OCEAN'),
 (0.007330429696632295, 'NEAR OCEAN'),
 (0.001987503501489285, 'NEAR BAY'),
 (5.510485338123466e-05, 'ISLAND')]

# Evaluating on Test System

In [55]:
# at this point, run the full_pipeline to transform the data -- call the transform() function, NOT the fit_transform -- YOU DO NOT WANT TO FIT THE TEST DATA
# we are noe evaluating the final model on the test set

final_model = grid_search.best_estimator_ 

X_test = strat_test_set.drop("median_house_value", axis = 1) # this contain the predictors / independent variables in the test dataset
y_test = strat_test_set["median_house_value"].copy()         # this contain the dependent variables in the test dataset

X_test_prepared = full_pipeline.transform(X_test)

In [56]:
final_predictions = final_model.predict(X_test_prepared)
final_mse = mean_squared_error(y_test, final_predictions)
final_rmse = np.sqrt(final_mse)
print(final_rmse)

47569.129976311844


# A Confidence Interval

In [57]:
# in some cases, a point estimate of the generalization error will not be quite enough 
# computing a 95% CI for the generalization error can help you to have an idea how precise this estimate is

from scipy import stats
confidence = 0.95

squared_errors = (final_predictions - y_test) ** 2
np.sqrt(stats.t.interval(confidence, len(squared_errors)-1, loc=squared_errors.mean(), scale = stats.sem(squared_errors)))

array([45578.87786644, 49479.39112239])